In [1]:
# Getting relevant packages
import xlrd
import pandas as pd
from collections import Counter 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import (figure, hold, plot, title, xlabel, ylabel, legend,
                               colorbar, imshow, xticks, yticks, show, savefig, boxplot)
%matplotlib inline

import graphviz
import subprocess
import sklearn.linear_model as lm

from pylab import *
from scipy.io import loadmat
from scipy.linalg import svd
from scipy import linalg, stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import model_selection, tree
from sklearn import model_selection
from operator import itemgetter
from sklearn.mixture import GaussianMixture
from scipy.stats.kde import gaussian_kde
from toolbox_02450 import gausKernelDensity
from sklearn.neighbors import NearestNeighbors
from toolbox_02450 import clusterplot

# Needed for Apriori
import sklearn.preprocessing as pr
from subprocess import call
import re
import os

### Needed function for Apriori

In [2]:
def cfor(first,test,update):
    while test(first):
        yield first
        first = update(first)

def WriteAprioriFile(X,titles=None,filename="AprioriFile.txt"):
	#Setup
	output = u"";
	N, M = np.shape(X);

	#Go through conversion loop
	if titles==None:
		for i in cfor(0,lambda i:i<N,lambda i:i+1):
			for j in cfor(0,lambda j:j<M,lambda j:j+1):
				if (X[i,j] == 1):
					output = output + str(j) + u",";
			output = output[:-1] + u"\n";		
	else:
		for i in cfor(0,lambda i:i<N,lambda i:i+1):
			for j in cfor(0,lambda j:j<M,lambda j:j+1):
				if (X[i,j] == 1):
					output = output + titles[j] + u",";
			output = output[:-1] + u"\n";
	
	#White output to file
	f = open(filename, 'w');
	f.write(output);
	f.close;

### Read in data

In [3]:
# Fetch sheet from xlsx file
#genera = pd.read_excel('dtu_compute_exercise_EFFORT_alldata.xlsx', sheetname='ResFinder_gene_counts')
genera = pd.read_excel('gene_output.xlsx')

# Transpose dataframe to get correct labeling of observations
#genera = genera.transpose()
del genera['clust_name']

In [4]:
# Extract column names
#genera = genera[1:]

# Print first 10 columns to visualize data
genera[:10]

,aac(2)-I,aac(3)-I,aac(3)-Ib-aac(6)-Ib,aac(3)-II,aac(3)-IV,aac(3)-VI,aac(3)-VII,aac(3)-X,aac(6),aac(6)-31,...,vat(C),vat(D),vat(E),vat(F),vga(A),vga(B),vga(C),vga(E),vgb(A),vgb(B)
0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,9,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
7,0,0,0,0,3,1,0,0,0,0,...,0,0,7,0,0,0,0,0,0,0
8,0,0,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


### Make data ready for association mining

In [4]:
#bin_data = pr.binarize(genera)

#writer = pd.ExcelWriter('gene_bin.xlsx')
#pd.DataFrame(X).to_excel(writer)
#writer.save()

In [5]:
# For association mining, normalize data as matrix
#genera_mat = genera.as_matrix()
#genera_mat = (genera_mat - np.ones((len(genera_mat),1))*genera_mat.mean(0))/genera_mat.std(0)

# Binarize data and save as matrix X
X = pr.binarize(genera)
X = np.mat(X)

# Make data ready for apriori - format string input
temp = ""
for i in range(0,len(X)):
    for j in range(0,len(genera.columns)):
        temp = temp + str(int(X[i,j])) + " "
    temp = temp.strip() + "; "

# Write data to file on "matlab form"
WriteAprioriFile(np.mat(temp[:-2]),filename="AprioriFor02582.txt")
#titles=list(genera.columns)

### Run Apriori algorithm

In [ ]:
filename = 'AprioriFor02582.txt'
minSup = 50
minConf = 100
maxRule = 5

# Run Apriori Algorithm
print('Mining for frequent itemsets by the Apriori algorithm')
status1 = call('Tools/apriori.exe -f"," -s{0} -v"[Sup. %S]" {1} apriori_temp1.txt'.format(minSup, filename))
if status1!=0:
    print('An error occured while calling apriori, a likely cause is that minSup was set too high such that no frequent itemsets were generated or spaces are included in the path to the apriori files.')
    exit()
if minConf>0:
    print('Mining for associations by the Apriori algorithm')
    status2 = call('Tools/apriori.exe -tr -f"," -n{0} -c{1} -s{2} -v"[Conf. %C,Sup. %S]" {3} apriori_temp2.txt'.format(maxRule, minConf, minSup, filename))
    if status2!=0:
        print('An error occured while calling apriori')
        exit()
print('Apriori analysis done, extracting results')


# Extract information from stored files apriori_temp1.txt and apriori_temp2.txt
f = open('apriori_temp1.txt','r')
lines = f.readlines()
f.close()
# Extract Frequent Itemsets
FrequentItemsets = ['']*len(lines)
sup = np.zeros((len(lines),1))
for i,line in enumerate(lines):
    FrequentItemsets[i] = line[0:-1]
    sup[i] = re.findall(' [-+]?\d*\.\d+|\d+]', line)[0][1:-1]
os.remove('apriori_temp1.txt')
    
# Read the file
f = open('apriori_temp2.txt','r')
lines = f.readlines()
f.close()
# Extract Association rules
AssocRules = ['']*len(lines)
conf = np.zeros((len(lines),1))
for i,line in enumerate(lines):
    AssocRules[i] = line[0:-1]
    conf[i] = re.findall(' [-+]?\d*\.\d+|\d+,', line)[0][1:-1]
os.remove('apriori_temp2.txt')    

# sort (FrequentItemsets by support value, AssocRules by confidence value)
AssocRulesSorted = [AssocRules[item] for item in np.argsort(conf,axis=0).ravel()]
AssocRulesSorted.reverse()
FrequentItemsetsSorted = [FrequentItemsets[item] for item in np.argsort(sup,axis=0).ravel()]
FrequentItemsetsSorted.reverse()
    
# Print the results
import time; time.sleep(.5)    
print('\n')
print('RESULTS:\n')
print('Frequent itemsets:')
for i,item in enumerate(FrequentItemsetsSorted):
    print('Item: {0}'.format(item))
print('\n')
print('Association rules:')
for i,item in enumerate(AssocRulesSorted):
    print('Rule: {0}'.format(item))


Mining for frequent itemsets by the Apriori algorithm
